<a href="https://colab.research.google.com/github/Norrr3r/ET-ANER/blob/main/1_Test_for_Majority_and_Weighted_voting_ANERCorp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install seqeval==1.2.2

In [9]:
import os
from sklearn.metrics import accuracy_score
#from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score



In [10]:
from seqeval.metrics import (
    classification_report,
    precision_score,
    recall_score,
    f1_score as f1,
    accuracy_score)
from seqeval.scheme import IOB2


In [11]:
def combine_predictions_for_hard_voting (files):
 #out = open("out.txt", "r+")
 bst='\t'
 list1 = [[] for h in range(25950)]
 dic ={}


 i=0  #index for files
 for file in files:
    #index = 3
    with open( file, 'r', encoding='utf-8') as f:
        k=0   #index for lines
        for line in f:

            if not line.strip():
              list1[k].append([''])

            else:
                line = line.strip().split()
                token, gold_tag,predicted_tag = line[0], line[1],line[2]
                if i==0:
                   list1[k].append([token])
                   list1[k].append([gold_tag])
                   list1[k].append([predicted_tag])
                #else:
                 #   list1[k].append([predicted_tag])

                elif any(token in sublist for sublist in list1[k]):
                   list1[k].append([predicted_tag])
                else:
                   dic[token] = predicted_tag

            k+=1
    i+=1

 for token, tag in dic.items():
       for i, sublist in enumerate(list1):
            if any(token in sublist for sublist in list1[i]):
               list1[i].append([tag])
               break


 with open('/content/Predictions_test.txt', 'w', encoding='utf-8') as f:
    for kline in list1:
        for item in kline:
            f.write( bst.join(item) + '\t')
        f.write('\n')
#    for token , ptags in dic.items():
 #    f.write( f"{token} \t {ptags} \n")

 file_name = '/content/Predictions_test.txt'
 return file_name



In [12]:
def Hard_Voting (files):
 predicted_tags = []
 vote_counts = {}
 list2=[]
 file_name = combine_predictions_for_hard_voting(files)
 with open(file_name, "r", encoding="utf-8") as f:
   for line in f:
            line = line.strip().split()

            if len(line)==0:
              continue

            elif len(line)==3:
              token = line[0]
              gold_tag = line[1]
              pred_tag = line[2]
              list2.append((token ,gold_tag, pred_tag))

            elif len(line) >3:
              token = line[0]
              gold_tag = line[1]

              for tag in line[2:]:
                if tag not in vote_counts:
                   vote_counts[tag] = 0
                vote_counts[tag] += 1
              pred_tag = max(vote_counts, key=vote_counts.get)
              list2.append((token ,gold_tag, pred_tag))
              vote_counts.clear()


 with open('Compined_Predictions.txt', 'w', encoding='utf-8') as f:
    for token, gold_tag, pred_tag in list2:
        f.write(f"{token}\t{gold_tag}\t{pred_tag}\n")

 file_voting = 'Compined_Predictions.txt'

 return file_voting

In [13]:
def Allmetrics_hard_voting (files):
 for file in files:
  with open(file, 'r') as f:
    data = f.read().strip().split('\n')

  sentences_gold_tag = []
  sentences_pred_tag = []
  gold_tags = []
  pred_tags = []

  for line in data[1:]:
    if not line.strip():
        # Add the current sentence's gold and pred tags to the list of all sentences
        sentences_gold_tag.append(gold_tags)
        sentences_pred_tag.append(pred_tags)
        # Reset the gold_tags and pred_tags lists for the next sentence
        gold_tags = []
        pred_tags = []
    else:
        token, gold_tag, pred_tag = line.strip().split('\t')
        gold_tags.append(gold_tag)
        pred_tags.append(pred_tag)

# Add the gold and pred tags for the last sentence to the list of all sentences
  sentences_gold_tag.append(gold_tags)
  sentences_pred_tag.append(pred_tags)

# Compute precision, recall, and F1-score
  precision= precision_score(sentences_gold_tag, sentences_pred_tag, average='micro' , scheme=IOB2)
  recall= recall_score(sentences_gold_tag, sentences_pred_tag, average='micro' , scheme=IOB2)
  f1_score= f1(sentences_gold_tag, sentences_pred_tag, average='micro', scheme=IOB2 )
  report = classification_report(sentences_gold_tag, sentences_pred_tag, scheme=IOB2, digits=4)



# Print the results
  print ('\n*************\n', file )

  print(f"Precision: {precision:.3f}")
  print(f"Recall: {recall:.3f}")
  print(f"F1-score: {f1_score:.3f}")

  #print (report)

#Hard Voting for test set

In [14]:

New_files = [ #"/content/predictions_ANERtestC10_Arbert.txt",
             # "/content/predictions_ANERtestC10_Arabert.txt",
        # "/content/predictions_ANERtestC10_MARBERT.txt",
     # "/content/predictions_ANERtestC10_Arabicner.txt",
     #  "/content/predictions_ANERtestC10_Qarib.txt",
     #  "/content/predictions_ANERtestC10_Asafya.txt",
     #  "/content/predictions_ANERtestC10_hatimimoh.txt",
          "/content/predictions_ANERtestC10_Camel-msa-ner.txt",
       "/content/predictions_ANERtestC10_Camel-ca-ner.txt",
         "/content/predictions_ANERtestC10_Camel-mix-ner.txt",
      "/content/predictions_ANERtestC10_Multilingual2.txt",
         ]


file_vote_predictions = Hard_Voting(New_files)

New_files.append(file_vote_predictions)


Allmetrics_hard_voting(New_files)


*************
 /content/predictions_ANERtestC10_Camel-msa-ner.txt
Precision: 0.916
Recall: 0.942
F1-score: 0.929

*************
 /content/predictions_ANERtestC10_Camel-ca-ner.txt
Precision: 0.872
Recall: 0.885
F1-score: 0.879

*************
 /content/predictions_ANERtestC10_Camel-mix-ner.txt
Precision: 0.916
Recall: 0.932
F1-score: 0.924

*************
 /content/predictions_ANERtestC10_Multilingual2.txt
Precision: 0.923
Recall: 0.950
F1-score: 0.936

*************
 Compined_Predictions.txt
Precision: 0.928
Recall: 0.946
F1-score: 0.937


#Soft Voting

In [15]:
def combine_files_for_soft_voting (files):
 list1 = [[] for h in range(25950)]
 dic ={}


 i=0
 for file , weight in files:
    #index = 3
    with open(file, 'r', encoding='utf-8') as f:
        k=0
        for line in f:

            if not line.strip():
              list1[k].append([''])

            else:
                line = line.strip().split()
                token, gold_tag,predicted_tag = line[0], line[1],line[2]
                if i==0:
                   list1[k].append([token])
                   list1[k].append([gold_tag])
                   list1[k].append([predicted_tag , weight])

                elif any(token in sublist for sublist in list1[k]):
                   list1[k].append([predicted_tag , weight])
                else:
                   dic[token] = [predicted_tag, weight]

            k+=1
    i+=1

 for token, tag in dic.items():
        for i, sublist in enumerate(list1):
          if any(isinstance(sublist, list) and token in sublist for sublist in list1[i]):
               #if isinstance(tag, list):
               # list1[i].extend(tag)
               #else:
                list1[i].append(tag)
                break


 with open('Predictions_soft_hard.txt', 'w', encoding='utf-8') as f:
    for kline in list1:
        for item in kline:
          if isinstance(item, list):
            f.write(','.join(map(str, item)))
          else:
            f.write(str(item))
          f.write('\t')
        f.write('\n')
#    for token , ptags in dic.items():
 #    f.write( f"{token} \t {ptags} \n")

 file_name = '/content/Predictions_soft_hard.txt'

 return file_name




In [16]:
def Soft_voting(files):
 model_weight =[]
 predicted_tags = []
 vote_counts = {}
 list2=[]
 label_sums = {}
 file_name = combine_files_for_soft_voting(files)

 with open(file_name, "r", encoding="utf-8") as f:
   for line in f:

            line = line.strip().split()

            if len(line)==0:
              continue
            elif len(line) == 3:
              token = line[0]
              gold_tag = line[1]
              pred_tag = line[2]
              list2.append((token , gold_tag, pred_tag))

            elif len(line) > 3:
              token = line[0]
              gold_tag = line[1]
              for tag in line[2:]:
                values = tag.split(',')
                if len(values) !=2:
                  continue
                label , value = values
                if label not in vote_counts:
                   vote_counts[label] = 0
                vote_counts[label] += float(value)
              pred_tag = max(vote_counts, key=vote_counts.get)
              list2.append((token , gold_tag, pred_tag))
              vote_counts.clear()


 with open('Compined_Predictions_soft_hard.txt', 'w', encoding='utf-8') as f:
    for token, gold_tag, pred_tag in list2:
        f.write(f"{token}\t{gold_tag}\t{pred_tag}\n")

 file_name = '/content/Compined_Predictions_soft_hard.txt'

 return file_name

In [17]:
def Allmetrics_soft_voting (files):

 for file , weight in files:
  with open(file, 'r', encoding='utf-8') as f:
    data = f.read().strip().split('\n')

  sentences_gold_tag = []
  sentences_pred_tag = []
  gold_tags = []
  pred_tags = []

  for line in data[1:]:
    if not line.strip():
        # Add the current sentence's gold and pred tags to the list of all sentences
        sentences_gold_tag.append(gold_tags)
        sentences_pred_tag.append(pred_tags)
        # Reset the gold_tags and pred_tags lists for the next sentence
        gold_tags = []
        pred_tags = []
    else:
        token, gold_tag, pred_tag = line.strip().split('\t')
        gold_tags.append(gold_tag)
        pred_tags.append(pred_tag)

# Add the gold and pred tags for the last sentence to the list of all sentences
  sentences_gold_tag.append(gold_tags)
  sentences_pred_tag.append(pred_tags)

# Compute precision, recall, and F1-score
  precision= precision_score(sentences_gold_tag, sentences_pred_tag, average='micro' , scheme=IOB2)
  recall= recall_score(sentences_gold_tag, sentences_pred_tag, average='micro' , scheme=IOB2)
  f1_score= f1(sentences_gold_tag, sentences_pred_tag, average='micro', scheme=IOB2 )
  report = classification_report(sentences_gold_tag, sentences_pred_tag, scheme=IOB2, digits=4)



# Print the results
  print ('\n*************\n', file )

  print(f"Precision: {precision:.3f}")
  print(f"Recall: {recall:.3f}")
  print(f"F1-score: {f1_score:.3f}")

  #print (report)

In [18]:
#weighted for each model

files = [ #["/content/predictions_ANERtestC10_Arbert.txt",0.0],   #87%
           #  ["/content/predictions_ANERtestC10_Arabert.txt",0.0],   #82
           #["/content/predictions_ANERtestC10_MARBERT.txt",0.0],     #84
         #["/content/predictions_ANERtestC10_Arabicner.txt",0.0],   #86
         #["/content/predictions_ANERtestC10_Qarib.txt",0.0],      #84
         #["/content/predictions_ANERtestC10_Asafya.txt",0.0],    #84
         #["/content/predictions_ANERtestC10_hatimimoh.txt",0.0],    #83
         ["/content/predictions_ANERtestC10_Camel-msa-ner.txt", 0.30], #929
        ["/content/predictions_ANERtestC10_Camel-ca-ner.txt", 0.16],  #88
        ["/content/predictions_ANERtestC10_Camel-mix-ner.txt", 0.18],  #924
        ["/content/predictions_ANERtestC10_Multilingual2.txt",  0.36]  #936
         ]



file_vote_predictions = Soft_voting(files)

files.append([file_vote_predictions,0.0])


Allmetrics_soft_voting(files)




*************
 /content/predictions_ANERtestC10_Camel-msa-ner.txt
Precision: 0.916
Recall: 0.942
F1-score: 0.929

*************
 /content/predictions_ANERtestC10_Camel-ca-ner.txt
Precision: 0.872
Recall: 0.885
F1-score: 0.879

*************
 /content/predictions_ANERtestC10_Camel-mix-ner.txt
Precision: 0.916
Recall: 0.932
F1-score: 0.924

*************
 /content/predictions_ANERtestC10_Multilingual2.txt
Precision: 0.923
Recall: 0.950
F1-score: 0.936

*************
 /content/Compined_Predictions_soft_hard.txt
Precision: 0.938
Recall: 0.954
F1-score: 0.946


#New Code for Weighted voting (this code is ony for testing)


In [19]:
#weighted for each model

files = [ #["/content/predictions_ANERtestC10_Arbert.txt",0.0],   #87%
           #  ["/content/predictions_ANERtestC10_Arabert.txt",0.0],   #82
           #["/content/predictions_ANERtestC10_MARBERT.txt",0.0],     #84
         #["/content/predictions_ANERtestC10_Arabicner.txt",0.0],   #86
         #["/content/predictions_ANERtestC10_Qarib.txt",0.0],      #84
         #["/content/predictions_ANERtestC10_Asafya.txt",0.0],    #84
         #["/content/predictions_ANERtestC10_hatimimoh.txt",0.0],    #83
         ["/content/predictions_ANERtestC10_Camel-msa-ner.txt", 0.253272], #929
        ["/content/predictions_ANERtestC10_Camel-ca-ner.txt", 0.23964],  #88
        ["/content/predictions_ANERtestC10_Camel-mix-ner.txt", 0.251908],  #924
        ["/content/predictions_ANERtestC10_Multilingual2.txt",  0.25518]  #936
         ]




file_vote_predictions = Soft_voting(files)

files.append([file_vote_predictions,0.0])


Allmetrics_soft_voting(files)




*************
 /content/predictions_ANERtestC10_Camel-msa-ner.txt
Precision: 0.916
Recall: 0.942
F1-score: 0.929

*************
 /content/predictions_ANERtestC10_Camel-ca-ner.txt
Precision: 0.872
Recall: 0.885
F1-score: 0.879

*************
 /content/predictions_ANERtestC10_Camel-mix-ner.txt
Precision: 0.916
Recall: 0.932
F1-score: 0.924

*************
 /content/predictions_ANERtestC10_Multilingual2.txt
Precision: 0.923
Recall: 0.950
F1-score: 0.936

*************
 /content/Compined_Predictions_soft_hard.txt
Precision: 0.934
Recall: 0.951
F1-score: 0.942
